To copy this template: File -> Save a Copy in Drive

***DISCLAIMER**: In case of any discrepancy in the assignment instruction, please refer to the `PDF` document.*

# Problem 2 - Neural Network Training and Backpropagation

## 2.1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

# sigmoid()
def sigmoid(z):
    return 1 / (1 + np.exp(-2 * z))

## 2.2

In [2]:
# forward_propagate()
def forward_propagate(X, theta1, theta2, theta3):
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    a3 = np.insert(sigmoid(z3), 0, values=np.ones(m), axis=1)
    z4 = a3 @ theta3.T
    h = sigmoid(z4)

    return a1, z2, a2, z3, a3, z4, h

## 2.3

In [3]:
# cost() without regularization
def cost(params, input_size, hidden_size_1, hidden_size_2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size_1 * (input_size + 1)], (hidden_size_1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1):hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1)], (hidden_size_2, (hidden_size_1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1):], (num_labels, (hidden_size_2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    return J

In [4]:
# cost() with regularization
def cost(params, input_size, hidden_size_1, hidden_size_2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size_1 * (input_size + 1)], (hidden_size_1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1):hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1)], (hidden_size_2, (hidden_size_1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1):], (num_labels, (hidden_size_2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))

    return J

## 2.4

In [5]:
# sigmoid_gradient()
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z))) * 2

## 2.5

In [6]:
# backprop() without regularization
def backprop(params, input_size, hidden_size_1, hidden_size_2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size_1 * (input_size + 1)], (hidden_size_1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1):hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1)], (hidden_size_2, (hidden_size_1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1):], (num_labels, (hidden_size_2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    delta3 = np.zeros(theta3.shape)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))

    # perform backpropagation
    for t in range(m):                
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        z3t = z3[t,:]
        a3t = a3[t,:]
        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt

        z3t = np.insert(z3t, 0, values=np.ones(1))
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(z3t))

        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(z2t))

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + (d3t[:,1:]).T * a2t
        delta3 = delta3 + d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad

In [7]:
# backprop() with regularization
def backprop(params, input_size, hidden_size_1, hidden_size_2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size_1 * (input_size + 1)], (hidden_size_1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1):hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1)], (hidden_size_2, (hidden_size_1 + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1):], (num_labels, (hidden_size_2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    delta3 = np.zeros(theta3.shape)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        z3t = z3[t,:]
        a3t = a3[t,:]
        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt

        z3t = np.insert(z3t, 0, values=np.ones(1))
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(z3t))

        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(z2t))

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + (d3t[:,1:]).T * a2t
        delta3 = delta3 + d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    delta3[:,1:] = delta3[:,1:] + (theta3[:,1:] * learning_rate) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad

## 2.6

In [8]:
# your code starts here
data = loadmat('data/ex3data1.mat')

X = data['X']
y = data['y']

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)

# initial setup
input_size = 400
hidden_size_1 = 20
hidden_size_2 = 20
num_labels = 10
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size_1 * (input_size + 1) + (hidden_size_1 + 1) * hidden_size_2 + num_labels * (hidden_size_2 + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

J, grad = backprop(params, input_size, hidden_size_1, hidden_size_2, num_labels, X, y_onehot, learning_rate)

from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size_1, hidden_size_2, num_labels, X, y_onehot, learning_rate),
                method='TNC', jac=True, options={'maxiter': 250})
fmin

C:\Users\as345\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\as345\AppData\Local\Temp\ipykernel_106512\1935732658.py:30: OptimizeWarning: Unknown solver options: maxiter
  fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size_1, hidden_size_2, num_labels, X, y_onehot, learning_rate),


 message: Linear search failed
 success: False
  status: 4
     fun: 0.10051665259208044
       x: [-2.550e-01  3.954e-05 ... -1.271e+00 -3.759e-01]
     nit: 43
     jac: [-3.500e-05  7.908e-09 ... -2.282e-05 -4.427e-06]
    nfev: 889

## 2.7

In [9]:
# your code starts here
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size_1 * (input_size + 1)], (hidden_size_1, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size_1 * (input_size + 1):hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1)], (hidden_size_2, (hidden_size_1 + 1))))
theta3 = np.matrix(np.reshape(fmin.x[hidden_size_1 * (input_size + 1) + hidden_size_2 * (hidden_size_1 + 1):], (num_labels, (hidden_size_2 + 1))))

a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 100.0%


## 2.8

**Answer:**
